# Titanic: Machine Learning from Disaster

### To predict the survival of the passengers

This script uses a dataset that was taken from kaggle.com from the competition [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data) :


Variable descriptions of the data:

| Column      | Description  |
|-------------|---|
|  survival   | Survival (0 = No; 1 = Yes)  |
|  pclass     |  Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)|
|  name       |  Name |
|  sex        |  Sex |
|  age        |  Age |
|  sibsp      | Number of Siblings/Spouses Aboard  |
|  parch      | Number of Parents/Children Aboard  |
|  ticket     | Ticket Number  |
|  fare       | Passenger Fare  |
|  cabin      |  Cabin  |
|  embarked   | Port of Embarkation (C=Cherbourg; Q=Queenstown; S=Southampton)|

Special notes:     

**Pclass** is a proxy for socio-economic status (SES) :
 * 1st ~ Upper; 
 * 2nd ~ Middle; 
 * 3rd ~ Lower

**Age** is in Years; Fractional if Age less than One (1). If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch) some relations were ignored.  The following are the definitions used for sibsp and parch.

**Sibling**:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic

**Spouse**:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)

**Parent**:   Mother or Father of Passenger Aboard Titanic

**Child**:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins, nephews/nieces, aunts/uncles, and in-laws.  Some children travelled only with a nanny, therefore parch=0 for them.  As well, some travelled with very close friends or neighbors in a village, however, the definitions do not support such relations.

In [1]:
import pandas as pd

train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

train_data[:3]

IOError: File Titanic/train.csv does not exist

## 1. Filling the values that are not available 

In [ ]:
import numpy as np
from pandas import DataFrame


def my_fillna(df):
    df['Cabin']=df['Cabin'].fillna('/')
    
    # 2 cases
    df['Embarked']=df['Embarked'].fillna('/')
    
    # it's only one case
    df['Fare']=df['Fare'].fillna( df['Fare'].median() )
    
    df['Parch']=df['Parch'].fillna( 0 )
    df['SibSp']=df['SibSp'].fillna( 0 )
    
my_fillna(train_data)    
my_fillna(test_data)

## 2. Splitting complex columns to separate different information

For example, I can split the column **Cabin** (ex 'C85') into 'Cabin letter' -which is the deck- ('C') and the 'Room number' ('85')
 

In [ ]:
def getCabinNumber(s):
    x= s.split(' ')[0][1:]
    if(len(x) ==0):
        return 0
    else:
        return int(x)

def getFirstChar(s):
    return s[0] if(len(s)>0) else '/'

def splitColumns(df):    
    df['CabinLetter']=df['Cabin'].apply(lambda s: getFirstChar(s))
    df['CabinNumber']=df['Cabin'].apply(lambda x: getCabinNumber(x))
    # there can be multiple cabins under a single name
    # TODO find better ways of extracting features
    df['NrCabins']=df['Cabin'].apply(lambda x: 0 if(x=='/') else len(x.split(' ')))
    
    df['Apelative']=df['Name'].apply(lambda x : x.split(' ')[1])
    df['LastName']=df['Name'].apply(lambda x : x.split(',')[0])    
    df['FirstName']=df['Name'].apply(lambda x : x.split(' ')[2])
    
    df['TicketSeria']=df['Ticket'].apply(lambda x: 
                                         hash(x.split(' ')[0]) if(' ' in x) else -1)
    
splitColumns(train_data)
splitColumns(test_data) 

### 3. Converting strings to numbers 

In [ ]:
def getSex(s):
    if(s == "male"): return 1
    else:
        if(s == "female"): return 0
        else: return 2 #unknown

def convertData(df):
    df['CabinLetter']=hash(df['CabinLetter'].to_string())
    #df['CabinNumber']=ord(df['CabinNumber'][0])
    df['Apelative']=hash(df['Apelative'].to_string())
    df['LastName']=hash(df['LastName'].to_string())    
    df['FirstName']=hash(df['FirstName'].to_string())
    df['Embarked']=hash(df['Embarked'].to_string())
    df['Sex']=df['Sex'].apply(lambda s: getSex(s))

convertData(train_data)
convertData(test_data)    

## 4. Inferring Age column

A lot of age values are missing (177 from 891 values). Since age is an important feature for determining the fate of a passenger (children and women had priority for the rescue boats) I will try to predict ages missing.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

def getAllAgeFeatures(df):
    return df[['Parch', 'SibSp']]
    
# ~700 rows
labeled_data = train_data[ train_data['Age'].notnull() ]

nr=10    
rf_sum=0
mean_sum=0
kf = KFold(n_splits=nr)
for train_index, test_index in kf.split(labeled_data):

    x_train=getAllAgeFeatures(labeled_data.iloc[train_index])
    x_test=getAllAgeFeatures(labeled_data.iloc[test_index])
  
    y_train=labeled_data.iloc[train_index]['Age']
    y_test=labeled_data.iloc[test_index]['Age']
    
    model = RandomForestRegressor().fit(x_train, y_train)
    
    predicted_age=model.predict(x_test)
    rf_sum=rf_sum + ((predicted_age - y_test)**2).sum()
    
    mean=labeled_data.iloc[train_index]['Age'].mean()
    mean_sum=mean_sum + ((mean - y_test)**2).sum()

    
print 'Cost function is Least-Squares'     
print 'Using random forest gressor: '+str(rf_sum/nr)    
print 'Using the mean value: '+str(mean_sum/nr)

In [ ]:
def predict_age(df):
    
    labeled_data = df[ df['Age'].notnull() ]    
    
    unlabeled_i=df['Age'].isnull()
    unlabeled_data = df[ unlabeled_i ]
    
    x_train=getAllAgeFeatures(labeled_data)
    y_train=labeled_data['Age']

    model = RandomForestRegressor().fit(x_train, y_train)

    df.loc[unlabeled_i, 'Age']=model.predict(getAllAgeFeatures(unlabeled_data))
    
predict_age(train_data)
predict_age(test_data)

## 5. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def get_features():
    return ["Pclass", "Fare", "Sex", "Age", "SibSp", "Parch", "NrCabins","CabinLetter"]
    
def get_df_features(df):
    return df[get_features()].as_matrix()
    
def train_model(X, y):
    rfc = RandomForestClassifier()
    rfc = rfc.fit(X, y)
    #print sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), get_features()), 
    #         reverse=True)
    return rfc
    

X=get_df_features(train_data)
y=train_data['Survived']

nr=10
cost=0
cost_2=0
kf = KFold(n_splits=nr)
for train_index, test_index in kf.split(X):
    
    model=train_model(X[train_index], y[train_index])
    
    predicted=model.predict(X[test_index])    
    cost=cost+((predicted - y[test_index])**2).sum()    
    
print 'Using random forest: successful_predictions/nr_cases: '
print str( (len(X) - cost)  / (float)(len(X)) )

In [ ]:
# train using all the labeled data
model = train_model(X, y) 

output=DataFrame()
output['PassengerId']=test_data['PassengerId']
output['Survived']=model.predict(get_df_features(test_data))


output[['PassengerId', 'Survived']].to_csv(path_or_buf='predictions.csv', header=True, index=False)
print 'Done.'